# OpenAi
> Experiments using the openAI library

- hide: true

In [ ]:
https://platform.openai.com/docs/guides/function-calling

In [ ]:
response = openai.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=messages,
    tools=tools,
    tool_choice="auto",  # auto is default, but we'll be explicit
    response_format={ type: "json_object" },
)

To use JSON mode, your system message must instruct the model to produce JSON. 
To help ensure you don't forget, the API will throw an error if the string "JSON" does not appear in your system message.

In [ ]:
from pathlib import Path
from openai import OpenAI
client = OpenAI(api_key="")

speech_file_path = "speech.mp3"
response = client.audio.speech.create(
  model="tts-1",
  voice="echo",
  input="""This is where I leave my notes (((I made this place for myself but it's open to the public.)))

Employer? README. 
 """
)

response.stream_to_file(speech_file_path)

In [ ]:
import openai
import json
openai.api_key = "sk- ..."
# Example dummy function hard coded to return the same weather
# In production, this could be your backend API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": location, "temperature": "10", "unit": "celsius"})
    elif "san francisco" in location.lower():
        return json.dumps({"location": location, "temperature": "72", "unit": "fahrenheit"})
    else:
        return json.dumps({"location": location, "temperature": "22", "unit": "celsius"})

def run_conversation():
    # Step 1: send the conversation and available functions to the model
    messages = [{"role": "user", "content": "What's the weather like in San Francisco, Tokyo, and Paris?"}]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            },
        }
    ]
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo-1106",
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    # Step 2: check if the model wanted to call a function
    if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "get_current_weather": get_current_weather,
        }  # only one function in this example, but you can have multiple
        messages.append(response_message)  # extend conversation with assistant's reply
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                location=function_args.get("location"),
                unit=function_args.get("unit"),
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response
        second_response = openai.chat.completions.create(
            model="gpt-3.5-turbo-1106",
            messages=messages,
        )  # get a new response from the model where it can see the function response
        return second_response
print(run_conversation())

In [ ]:
#hide 
// console.log(OpenAI.Audio.Speech);  // Audio Speech // Assistant // Completions + Functions
  
  // thread Id =  
  // run Id =  
  
  /*
  const thread = await openai.beta.threads.create();
  console.log('thread', thread.id);
  
  // Add a Comment to a Thread
  const message = await openai.beta.threads.messages.create( thread.id, { role: "user", content: "I need to solve the equation `3x + 11 = 14`. Can you help me?" } );
  
  // Add Chatbot with Thread
  const run = await openai.beta.threads.runs.create( thread.id, { assistant_id: "asst_md7rq9HME4C3uUnmrMwRqiDi", instructions: "Please address the user as Jane Doe. The user has a premium account." } );
  console.log('run', run.id);
  */
  /*
  const run = await openai.beta.threads.runs.retrieve(
      "thread_YkCHeyvt4cpTx3dsRCAXamji",
      "run_dXteBI7ldfWlJ27tlaQXygVQ"
  );
  const messages = await openai.beta.threads.messages.list( "thread_YkCHeyvt4cpTx3dsRCAXamji" );
  console.log(messages);
  */